In [8]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
import os


class MaskDetectorAndHandTracker:
    def __init__(self, model_path):
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Model tidak ditemukan di: {model_path}")
        
        self.mp_face_detection = mp.solutions.face_detection
        self.mp_hands = mp.solutions.hands
        self.mp_drawing = mp.solutions.drawing_utils
        
        self.face_detection = self.mp_face_detection.FaceDetection(min_detection_confidence=0.9)
        self.hands = self.mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

        try:
            self.model = load_model(model_path)
            dummy_data = np.zeros((1, 224, 224, 3))
            self.model.predict(dummy_data, verbose=0)
        except Exception as e:
            raise Exception(f"Error saat loading model: {str(e)}")
        
        self.classes = ['with_mask', 'without_mask']
        self.confidence_threshold = 0.9

    def preprocess_face(self, face_img):
        if face_img is None or face_img.size == 0:
            print("Error: Wajah kosong atau tidak valid")
            return None
        face_img = cv2.resize(face_img, (224, 224))
        face_img = face_img.astype('float32') / 255.0
        return np.expand_dims(face_img, axis=0)

    def get_face_boxes(self, face_detection_result, image):
        if face_detection_result is None or not face_detection_result.detections:
            return []

        image_height, image_width, _ = image.shape
        boxes = []
        for detection in face_detection_result.detections:
            bbox = detection.location_data.relative_bounding_box
            x = max(0, int(bbox.xmin * image_width))
            y = max(0, int(bbox.ymin * image_height))
            w = min(int(bbox.width * image_width), image_width - x)
            h = min(int(bbox.height * image_height), image_height - y)
            if w > 0 and h > 0:
                boxes.append((x, y, w, h))
        return boxes

    def is_hand_over_face(self, face_box, hand_landmarks, image_width, image_height):
        """Cek apakah tangan berada di atas wajah."""
        x, y, w, h = face_box
        for landmark in hand_landmarks:
            landmark_x = int(landmark.x * image_width)
            landmark_y = int(landmark.y * image_height)
            if x <= landmark_x <= x + w and y <= landmark_y <= y + h:
                return True
        return False

    def detect_mask(self, image, hand_landmarks_list, image_width, image_height):
        if image is None or image.size == 0:
            print("Error: Gambar kosong atau tidak valid")
            return image
        
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = self.face_detection.process(image_rgb)
        boxes = self.get_face_boxes(results, image)
        
        for x, y, w, h in boxes:
            # Cek apakah ada tangan di atas wajah
            if any(self.is_hand_over_face((x, y, w, h), hand_landmarks, image_width, image_height) for hand_landmarks in hand_landmarks_list):
                continue  # Abaikan deteksi masker untuk wajah ini
            
            face = image[y:y+h, x:x+w]
            if face.size == 0:
                continue
            processed_face = self.preprocess_face(face)
            if processed_face is None:
                continue
            prediction = self.model.predict(processed_face, verbose=0)
            class_idx = np.argmax(prediction[0])
            confidence = prediction[0][class_idx]

            if confidence < self.confidence_threshold:
                class_idx = 1

            color = (0, 255, 0) if class_idx == 0 else (0, 0, 255)
            label = f'{self.classes[class_idx]}: {confidence:.2f}'
            cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)
            cv2.putText(image, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        return image

    def detect_hands(self, image):
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = self.hands.process(image_rgb)
        hand_landmarks_list = []
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                hand_landmarks_list.append(hand_landmarks.landmark)
                self.mp_drawing.draw_landmarks(
                    image, hand_landmarks, self.mp_hands.HAND_CONNECTIONS,
                    self.mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                    self.mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2)
                )
        return image, hand_landmarks_list


def main():
    try:
        model_path = "model_face_mask.keras"
        
        if not os.path.exists(model_path):
            print(f"Error: Model tidak ditemukan di {model_path}")
            return
        
        detector = MaskDetectorAndHandTracker(model_path)
        
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("Error: Tidak dapat membuka webcam")
            return
        
        print("Tekan 'q' untuk keluar")
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Tidak dapat membaca frame dari webcam")
                break

            image_height, image_width, _ = frame.shape
            frame, hand_landmarks_list = detector.detect_hands(frame)
            frame = detector.detect_mask(frame, hand_landmarks_list, image_width, image_height)
            
            cv2.imshow('Mask and Hand Detection', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()
    except Exception as e:
        print(f"Error dalam main: {str(e)}")
        if 'cap' in locals():
            cap.release()
        cv2.destroyAllWindows()


if __name__ == "__main__":
    main()


Tekan 'q' untuk keluar
